# For HBL tiles, get yearly Fmask products

In [1]:
import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import STAC_Utils as sutil
import PreProcess_Utils as pputil

import geopandas as gpd
import pandas as pd
import os.path
import traceback
import rioxarray

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59735,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59795,Total threads: 10
Dashboard: http://127.0.0.1:59806/status,Memory: 0 B
Nanny: tcp://127.0.0.1:59738,


In [3]:
# Prepare area
area = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\UtilityData\Study Areas\Canada\TileSystems\60km\canadaFishnet_CanVec_60km.shp"
area = gpd.read_file(area, engine = 'pyogrio').drop(columns = ['Year', 'Download', 'Complete'])
area = area[area['HBL'] == 1]
area

,HBL,PERCENTAGE,geometry
365,1.0,100.000000,"POLYGON ((588972.823 175276.489, 588972.823 23..."
366,1.0,100.000000,"POLYGON ((648972.823 175276.489, 648972.823 23..."
367,1.0,100.000000,"POLYGON ((708972.823 175276.489, 708972.823 23..."
368,1.0,100.000000,"POLYGON ((768972.823 175276.489, 768972.823 23..."
369,1.0,100.000000,"POLYGON ((828972.823 175276.489, 828972.823 23..."
...,...,...,...
1472,1.0,17.175259,"POLYGON ((108972.823 1015276.489, 108972.823 1..."
1542,1.0,100.000000,"POLYGON ((-71027.177 1075276.489, -71027.177 1..."
1543,1.0,67.768568,"POLYGON ((-11027.177 1075276.489, -11027.177 1..."
1544,1.0,15.244962,"POLYGON ((48972.823 1075276.489, 48972.823 113..."


In [4]:
roi = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\UtilityData\Study Areas\Canada\Boundary\CanVec\CanVec_Dissolve3979_Buffer10km_Erase_Edit_FillHoles.shp'
roi = gpd.read_file(roi, engine = 'pyogrio')
roi

,ctry_en,ORIG_FID,geometry
0,Canada,0,"MULTIPOLYGON (((2720695.69 188052.83, 2720041...."


In [5]:
# Prepare other variables
year = 2022 # 2018, 2019, 2020, 2021, 2022, 2023, 2024
start = str(year) + '-01-01'
end =  str(year) + '-12-31'
bands = ['Fmask']
chunksize = (1, -1, -1, -1)
sceneCloudThresh = 100
proj = 'poly'
verbose = True

In [6]:
# Access NASA Earthdata STAC
gdalEnv, catalog = sutil.accessSTAC('https://cmr.earthdata.nasa.gov/stac/LPCLOUD')

In [7]:
# For each tile...
for tile in range(len(area)):

    try: 

        id = area.index[tile]

        print('--------------------')
        print('...Tile ' + str(tile + 1) + ' (ID: ' + str(id) + ')' + ' of ' + str(len(area)) + '...')
    
        # Get ID for saving and confirm tile and year not already saved    
        nc = 'fmask_' + str(id) + '_' + str(year) + '.nc'
        path = 'F:/HLS/Canada/Tiles60km/fmask/' + str(year)
        if os.path.isfile(path + '/' + nc):
            print('Tile data already saved as NetCDF.')
            continue
    
        # Build HLS Fmask cube from STAC
        fmask = sutil.buildHLS(area.iloc[[tile]], start, end, bands, chunksize, sceneCloudThresh, proj, False, gdalEnv, catalog, verbose)
    
        # Load into memory
        fmask = pputil.loadXR(fmask)
        print('Fmask cube loaded into memory.')
    
        # Convert Fmask
        fmask = pputil.convertFmask(fmask, format = 'categories')
        print('Fmask cube converted into categories.')

        # Border cleaning
        if area.iloc[tile]['PERCENTAGE'] < 100:
            fmask = fmask.rio.clip(roi.geometry.values, drop = False) # Sets outside border to 0 for uint8
            print('Observations outside ROI (' + str(round(100 - area.iloc[tile]['PERCENTAGE'], 3)) + '%) masked (value 0).')
    
        # Download as NetCDF
        pputil.downloadNC(fmask, path, nc, form = 'hls')
        print('Fmask cube downloaded as NetCDF: ' + path + '/' + nc + '.')
    
        print('--------------------')

    except Exception as error: # Server errors every so often...
        #print('Skipping tile ' + str(tile) + ' (', type(error).__name__, ').') # Brief error type reporting (API or CPLE for server related)
        print(traceback.format_exc()) # Gives more detail about error (for debugging)
        continue  
    

--------------------
...Tile 1 (ID: 365) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 2 (ID: 366) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 3 (ID: 367) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 4 (ID: 368) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 5 (ID: 369) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 6 (ID: 370) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 7 (ID: 438) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 8 (ID: 439) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 9 (ID: 440) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 10 (ID: 441) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 11 (ID: 442) of 160...
Tile data already saved as NetCDF.
--------------------
...Tile 12 (ID: 443) of 160...
